In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
train_df=pd.read_csv('./output/train_hypothesis_evidences.csv')
train_df

,hypothesis,evidences,label
0,All the primary trial participants do not rece...,"[""INTERVENTION 1: "", "" Diagnostic (FLT PET)"",...",Contradiction
1,"Patients with Platelet count over 100,000/mm¬¨...","["" PATIENT CHARACTERISTICS:"", "" ANC 1,500/m...",Contradiction
2,Heart-related adverse events were recorded in ...,"[""Adverse Events 1:"", "" Supraventricular tach...",Entailment
3,Adult Patients with histologic confirmation of...,"[""Inclusion Criteria:"", "" Patients with histo...",Contradiction
4,Laser Therapy is in each cohort of the primary...,"[""INTERVENTION 1: "", "" Laser Therapy Alone"", ...",Contradiction
...,...,...,...
1695,"Adequate blood, kidney, and hepatic function a...","[""Inclusion Criteria:"", "" Postmenopausal wome...",Entailment
1696,The Ridaforolimus + Dalotuzumab + Exemestane g...,"[""Outcome Measurement: "", "" 1. Progression-fr...",Contradiction
1697,The only difference between the interventions ...,"[""INTERVENTION 1: "", "" Prone"", ""Prone positio...",Entailment
1698,Patients must have a white blood cell count ab...,"["" WBC > 1,500/mm\u00b3""]",Entailment


In [3]:
hypothesis_lst=train_df['hypothesis'].values.tolist()
len(hypothesis_lst)

1700

In [4]:
evidence_lst=train_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

1700

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=train_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

1700

In [6]:
class InputSequence:
    
    def __init__(self,tok,l_text,l_text2,l_label,batch_size=64,gpu=True):
        
        self.data_len=len(l_text)
        self.data_idx=[i for i in range(self.data_len)]
        self.texts=tok(l_text,l_text2,padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label=np.array(l_label)
        print('tokenize done')
        
        self.batch_size=batch_size
        self.gpu=gpu
        
    def on_epoch_end(self):
        random.shuffle(self.data_idx)
        
    def __getitem__(self,i):
        start=i*self.batch_size
        batch_idx=self.data_idx[start:min(start+self.batch_size,self.data_len)]
        
        return_texts=dict([(k,self.texts[k][batch_idx]) for k in self.texts])
        return_labels=torch.from_numpy(
            self.l_label[batch_idx].astype(np.int64)
        )
        
        if self.gpu:
            return_texts=dict([(k,return_texts[k].cuda()) for k in return_texts])
            return_labels=return_labels.cuda()
        
        return return_texts,return_labels
    
    def __len__(self):
        return math.ceil(1.0*self.data_len/self.batch_size)
    

In [7]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, MegatronBertForSequenceClassification

In [8]:
# text_tok=AutoTokenizer.from_pretrained('bert-base-uncased')
# text_clf=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
#roberta-base
# text_tok=AutoTokenizer.from_pretrained('roberta-base')
# text_clf=AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=2)
#bio-bert
# text_tok=AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
# text_clf=AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1',num_labels=2)
#biomegatron
text_tok=BertTokenizer.from_pretrained('../transformer_models/biomegatron345uncased')
text_clf=MegatronBertForSequenceClassification.from_pretrained('../transformer_models/biomegatron345uncased',num_labels=2)

Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at ../transformer_models/biomegatron345uncased and are newly initialized: ['classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)
len(training_data)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

tokenize done


27

In [10]:
class Model(nn.Module):
    def __init__(self,clf):
        super(Model, self).__init__()
        self.clf=clf
        self.loss=nn.CrossEntropyLoss()
    
    def forward(self, texts, labels, gpu=True):
        
        loss=self.loss(self.clf(**texts).logits, labels)
        
        return loss

In [11]:
model=Model(text_clf)

In [12]:
bat_s=16
l_rate=1e-5

training_data.batch_size=bat_s

model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)
total_epoch_num=10
for epoch in range(total_epoch_num):
    training_data.on_epoch_end()
    loss_sum=0.0
    loss_count=0
    for batch in range(len(training_data)):
        optimizer.zero_grad()
        batch_texts,batch_labels=training_data[batch]
        loss_count+=len(batch_texts['input_ids'])
        loss = model(
            batch_texts,batch_labels
        )
        print('epoch:',epoch,'batch:',batch,'loss:',loss.item(),end='\n' if batch==0 or batch+1==len(training_data) or (batch+1)%1000==0 else '\r')
        loss_sum += 1.0*loss.item()*len(batch_texts['input_ids'])
        loss.backward()
        optimizer.step()
    # model.clf.save_pretrained('./output/clf_models/bert-base-uncased_epoch_{}.pt'.format(format(epoch,'05d')))
    #roberta-base
    # model.clf.save_pretrained('./output/clf_models/roberta-base_epoch_{}.pt'.format(format(epoch,'05d')))
    #bio-bert
    # model.clf.save_pretrained('./output/clf_models/biobert-v1.1_epoch_{}.pt'.format(format(epoch,'05d')))
    #biomegatron
    model.clf.save_pretrained('./output/clf_models/biomegatron345uncased_epoch_{}.pt'.format(format(epoch,'05d')))
_=model.cpu()

epoch: 0 batch: 0 loss: 3.237522602081299
epoch: 0 batch: 106 loss: 1.2001129388809204
epoch: 1 batch: 0 loss: 1.4357892274856567
epoch: 1 batch: 106 loss: 0.6446853876113892
epoch: 2 batch: 0 loss: 0.6489815711975098
epoch: 2 batch: 106 loss: 0.84655880928039556
epoch: 3 batch: 0 loss: 0.33457714319229126
epoch: 3 batch: 106 loss: 0.30442610383033757
epoch: 4 batch: 0 loss: 0.3997654318809509
epoch: 4 batch: 106 loss: 0.08616403490304947
epoch: 5 batch: 0 loss: 0.24224041402339935
epoch: 5 batch: 106 loss: 0.094084233045578235
epoch: 6 batch: 0 loss: 0.1728304773569107
epoch: 6 batch: 106 loss: 1.589559078216552705
epoch: 7 batch: 0 loss: 0.06692104041576385
epoch: 7 batch: 106 loss: 0.14038570225238879
epoch: 8 batch: 0 loss: 0.04239814355969429
epoch: 8 batch: 106 loss: 0.044924989342689514
epoch: 9 batch: 0 loss: 0.022869184613227844
epoch: 9 batch: 106 loss: 0.010261015035212045
